<a href="https://colab.research.google.com/github/lisa11323/CSR_Laddit/blob/main/Study_1_empath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Preprocessed data

Remove non-English characters + strip emojis + filter by syllable count

In [ ]:
!pip install -U pandas numpy
!pip install nltk emoji

In [ ]:
import pandas as pd
import re
import emoji
from tqdm import tqdm
from google.colab import files

# 📁 파일 업로드
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# ⏳ tqdm과 pandas 연동
tqdm.pandas()

# 📄 엑셀 파일 읽기
df = pd.read_excel(file_name)

# 🧹 이모지 + 비영어 문자 제거 함수
def clean_text(text):
    if isinstance(text, float):
        text = str(text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

# 🔄 전처리 적용 (기존 'Review' 덮어쓰기)
df['Review'] = df['Review'].progress_apply(clean_text)

# ✨ 리뷰 길이 계산 함수
def get_review_length(text):
    try:
        return len(str(text).split())
    except:
        return 0

# 📏 Review_length 컬럼 생성
df['Review_length'] = df['Review'].progress_apply(get_review_length)

# 🚫 리뷰 길이 10 미만 제거, 기준을 바꾸려면 여기에 있는 숫자 10을 수정
df = df[df['Review_length'] >= 10].reset_index(drop=True)

# ✅ 엑셀로 저장
preprocessed_file_path = 'preprocessed_comments_with_length.xlsx'
df.to_excel(preprocessed_file_path, index=False)
print(f"✅ 전처리 + 리뷰 길이 ≥10 필터링 데이터를 '{preprocessed_file_path}' 파일로 저장했습니다.")

# 📥 다운로드
files.download(preprocessed_file_path)

Extract the Top-N most frequent words from the Reddit comment data

In [ ]:
import pandas as pd
from collections import Counter
import re
from tqdm import tqdm
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_excel(filename)

if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 데이터에 없습니다!")

print(f"✅ 총 댓글 수: {len(df)}개")

# 모든 댓글을 하나의 문자열로 합치기
all_text = ' '.join(df['comment'].dropna().astype(str).tolist()).lower()

# 단어만 추출
words = re.findall(r'\b\w+\b', all_text)

# 단어 빈도 계산
word_counts = Counter(words)

# 상위 50개 단어 출력
top_n = 50
print("\n🔷 상위 단어 Top-50:")
for word, count in word_counts.most_common(top_n):
    print(f"{word}: {count}")

In [ ]:
# 불용어 제거
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

filtered_words = [w for w in words if w not in stop_words]

# 불용어 제거 후 단어 빈도 재계산
filtered_word_counts = Counter(filtered_words)

print("\n🔷 불용어 제거 후 상위 단어 Top-50:")
for word, count in filtered_word_counts.most_common(50):
    print(f"{word}: {count}")

## 1. Independent Variable

Trust

In [ ]:
# 📦 Install libraries
!pip install empath pandas tqdm openpyxl --quiet

# 📚 Import libraries
import pandas as pd
from empath import Empath
from tqdm import tqdm
from google.colab import files

# ⬆️ Upload data
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 Load data
df = pd.read_excel(filename)

# ✅ Check required column
if 'comment' not in df.columns:
    raise ValueError("❌ The 'comment' column is missing from the data!")
if df['comment'].isnull().all():
    raise ValueError("❌ The 'comment' column contains no data to analyze!")

print(f"✅ Total number of comments: {len(df)}")

# 🤖 Load Empath
lexicon = Empath()
print("🔷 The default 'trust' category in Empath is handled internally.")

# 🔷 Custom Reddit-specific trust word list
reddit_trust_words_full = [
    'vouch', 'got', 'back', 'solid', 'faithful', 'reliable', 'loyal',
    'stand', 'based', 'legit', 'support', 'believe', 'integrity',
    'ride', 'crew', 'dayone', 'fam', 'homie', 'kin',
    'respect', 'props', 'cover', 'cosign', 'trustworthy', 'dependable',
    'buddy', 'partner', 'ally', 'friend', 'assist', 'helpful'
]

# ✅ Remove duplicates & sort
reddit_trust_words_unique = sorted(set(reddit_trust_words_full))
print(f"🔷 Number of unique Reddit-specific trust words: {len(reddit_trust_words_unique)}")
print(reddit_trust_words_unique)

# 🔷 Create custom category
lexicon.create_category("reddit_trust", reddit_trust_words_unique)
print("✅ Custom category 'reddit_trust' created successfully!")

# 🔷 Verify that categories are properly registered (optional)
test_result = lexicon.analyze("this is a test", categories=["trust", "reddit_trust"])
if not all(cat in test_result for cat in ["trust", "reddit_trust"]):
    raise RuntimeError("❌ Either 'trust' or 'reddit_trust' category was not registered correctly!")

# 🧠 Analysis function: default trust + custom reddit_trust + combined
def analyze_and_combine_trust(text):
    result = lexicon.analyze(str(text), categories=["trust", "reddit_trust"], normalize=True)
    trust_score = result.get('trust', 0.0)
    reddit_trust_score = result.get('reddit_trust', 0.0)
    combined_trust = trust_score + reddit_trust_score
    return pd.Series({
        "trust_score": round(trust_score, 4),
        "reddit_trust_score": round(reddit_trust_score, 4),
        "combined_trust_score": round(combined_trust, 4)
    })

# 🏃 Run analysis
tqdm.pandas()
trust_df = df['comment'].progress_apply(analyze_and_combine_trust)

# 🔗 Combine results
result_df = pd.concat([df, trust_df], axis=1)

# 💾 Save results
output_file = filename.replace(".xlsx", "_trust_combined_empath_expanded.xlsx")
result_df.to_excel(output_file, index=False, engine='openpyxl')
files.download(output_file)

print("✅ Done: Metrics including the extended custom trust scores have been calculated and saved.")

social engagement

In [ ]:
# 📦 라이브러리 설치
!pip install empath pandas tqdm openpyxl --quiet

# 📚 라이브러리 불러오기
import pandas as pd
from empath import Empath
from tqdm import tqdm
from google.colab import files

# ⬆️ 데이터 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 필수 컬럼 체크
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 데이터에 없습니다!")
if df['comment'].isnull().all():
    raise ValueError("❌ 'comment' 열에 분석할 데이터가 없습니다!")

print(f"✅ 총 댓글 수: {len(df)}개")

# 🤖 Empath 로드
lexicon = Empath()

# 🔷 기존 Social Engagement 카테고리 단어 집합 가져오기
base_words = set(lexicon.cats['communication']) \
           | set(lexicon.cats['meeting']) \
           | set(lexicon.cats['social_media'])

print(f"✅ 기존 Social Engagement 단어 수: {len(base_words)}")

# 🔷 Reddit 특화 Social Engagement 단어 리스
reddit_social_engagement_words_full = [
    'subreddit', 'thread', 'reply', 'post', 'topic', 'comment', 'dm',
    'debate', 'discussion', 'poll', 'vote', 'feedback', 'engage',
    'interact', 'participate',
    'connect', 'bond', 'exchange', 'help', 'support',
    'mention', 'tag', 'like', 'share', 'follow', 'trend'
]

# 🔷 중복 제거 & 정렬
reddit_social_engagement_words_unique = sorted({
    word for word in reddit_social_engagement_words_full if word not in base_words
})

print(f"✅ 중복 제거 후 사용자 정의 단어 수: {len(reddit_social_engagement_words_unique)}")
print(reddit_social_engagement_words_unique)

# 🔷 사용자 정의 카테고리 생성
lexicon.create_category("reddit_social_engagement", reddit_social_engagement_words_unique)
print("✅ 사용자 정의 카테고리 'reddit_social_engagement' 생성 완료!")

# 🔷 카테고리 정상 등록 테스트
test_result = lexicon.analyze("this is a test post reply engage", categories=[
    "communication", "meeting", "social_media", "reddit_social_engagement"
])
expected_categories = ["communication", "meeting", "social_media", "reddit_social_engagement"]
if not all(cat in test_result for cat in expected_categories):
    raise RuntimeError("❌ 하나 이상의 카테고리가 올바르게 등록되지 않았습니다.")

# 🧠 분석 함수: 기존 + 사용자 정의 + 합산
def analyze_and_combine_social_engagement(text):
    result = lexicon.analyze(
        str(text),
        categories=["communication", "meeting", "social_media", "reddit_social_engagement"],
        normalize=True
    )
    base_score = sum([
        result.get('communication', 0.0),
        result.get('meeting', 0.0),
        result.get('social_media', 0.0)
    ])
    reddit_score = result.get('reddit_social_engagement', 0.0)
    combined_score = base_score + reddit_score
    return pd.Series({
        "social_engagement_base_score": round(base_score, 4),
        "reddit_social_engagement_score": round(reddit_score, 4),
        "combined_social_engagement_score": round(combined_score, 4)
    })

# 🏃 분석 실행
tqdm.pandas()
engagement_df = df['comment'].progress_apply(analyze_and_combine_social_engagement)

# 🔗 데이터 합치기
result_df = pd.concat([df, engagement_df], axis=1)

# 💾 결과 저장
output_file = filename.replace(".xlsx", "_social_engagement_combined_empath.xlsx")
result_df.to_excel(output_file, index=False, engine='openpyxl')
files.download(output_file)

print("✅ 완료: 기존 + 사용자 정의 Social Engagement 점수가 계산되어 저장되었습니다.")

Norms of reciprocity

In [ ]:
# 📦 라이브러리 설치
!pip install empath pandas tqdm openpyxl --quiet

# 📚 라이브러리 불러오기
import pandas as pd
from empath import Empath
from tqdm import tqdm
from google.colab import files

# ⬆️ 데이터 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 필수 컬럼 체크
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 데이터에 없습니다!")
if df['comment'].isnull().all():
    raise ValueError("❌ 'comment' 열에 분석할 데이터가 없습니다!")

print(f"✅ 총 댓글 수: {len(df)}개")

# 🤖 Empath 로드
lexicon = Empath()

# 🔷 기존 Reciprocity 카테고리 단어 집합 가져오기
base_words = set(lexicon.cats['help']) \
           | set(lexicon.cats['politeness']) \
           | set(lexicon.cats['sympathy'])

print(f"✅ 기존 Reciprocity 단어 수: {len(base_words)}")

# 🔷 최종 확장된 사용자 정의 리스트
reddit_reciprocity_words_final = [
    # … (너가 작성한 리스트 그대로)
]

# 🔷 중복 제거 & 정렬
reddit_reciprocity_words_unique = sorted({
    word for word in reddit_reciprocity_words_final if word not in base_words
})

print(f"✅ 중복 제거 후 사용자 정의 단어 수: {len(reddit_reciprocity_words_unique)}")
print(reddit_reciprocity_words_unique)

# 🔷 사용자 정의 카테고리 생성
lexicon.create_category("reddit_reciprocity", reddit_reciprocity_words_unique)
print("✅ 사용자 정의 카테고리 'reddit_reciprocity' 생성 완료!")

# 🔷 카테고리 정상 등록 테스트
test_result = lexicon.analyze("thank you for your help", categories=[
    "help", "politeness", "sympathy", "reddit_reciprocity"
])
expected_categories = ["help", "politeness", "sympathy", "reddit_reciprocity"]
if not all(cat in test_result for cat in expected_categories):
    raise RuntimeError("❌ 하나 이상의 카테고리가 올바르게 등록되지 않았습니다.")

# 🧠 분석 함수: 기존 + 사용자 정의 + 합산
def analyze_and_combine_reciprocity(text):
    result = lexicon.analyze(
        str(text),
        categories=["help", "politeness", "sympathy", "reddit_reciprocity"],
        normalize=True
    )
    base_score = sum([
        result.get('help', 0.0),
        result.get('politeness', 0.0),
        result.get('sympathy', 0.0)
    ])
    reddit_score = result.get('reddit_reciprocity', 0.0)
    combined_score = base_score + reddit_score
    return pd.Series({
        "reciprocity_base_score": round(base_score, 4),
        "reddit_reciprocity_score": round(reddit_score, 4),
        "combined_reciprocity_score": round(combined_score, 4)
    })

# 🏃 분석 실행
tqdm.pandas()
reciprocity_df = df['comment'].progress_apply(analyze_and_combine_reciprocity)

# 🔗 데이터 합치기
result_df = pd.concat([df, reciprocity_df], axis=1)

# 💾 결과 저장
output_file = filename.replace(".xlsx", "_reciprocity_combined_empath_final.xlsx")
result_df.to_excel(output_file, index=False, engine='openpyxl')
files.download(output_file)

print("✅ 완료: 최종 확장 사용자 정의 단어를 반영한 Reciprocity 점수가 계산되어 저장되었습니다.")

relational bonding

In [ ]:
# 📦 라이브러리 설치
!pip install empath pandas tqdm openpyxl --quiet

# 📚 라이브러리 불러오기
import pandas as pd
from empath import Empath
from tqdm import tqdm
from google.colab import files

# ⬆️ 데이터 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 필수 컬럼 체크
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 데이터에 없습니다!")
if df['comment'].isnull().all():
    raise ValueError("❌ 'comment' 열에 분석할 데이터가 없습니다!")

print(f"✅ 총 댓글 수: {len(df)}개")

# 🤖 Empath 로드
lexicon = Empath()

# 🔷 기존 Relational Bonding 카테고리 단어 집합 가져오기
base_words = set(lexicon.cats['leader']) | set(lexicon.cats['family'])
print(f"✅ 기존 Relational Bonding 단어 수: {len(base_words)}")

# 🔷 확장된 사용자 정의 리스트
reddit_relational_bonding_words_final = [
    # … (너가 작성한 리스트 그대로)
]

# 🔷 중복 제거 & 정렬
reddit_relational_bonding_words_unique = sorted({
    word for word in reddit_relational_bonding_words_final if word not in base_words
})

print(f"✅ 중복 제거 후 사용자 정의 단어 수: {len(reddit_relational_bonding_words_unique)}")
print(reddit_relational_bonding_words_unique)

# 🔷 사용자 정의 카테고리 생성
lexicon.create_category("reddit_relational_bonding", reddit_relational_bonding_words_unique)
print("✅ 사용자 정의 카테고리 'reddit_relational_bonding' 생성 완료!")

# 🔷 카테고리 정상 등록 테스트
test_result = lexicon.analyze("we are a team and I appreciate you", categories=[
    "leader", "family", "reddit_relational_bonding"
])
expected_categories = ["leader", "family", "reddit_relational_bonding"]
if not all(cat in test_result for cat in expected_categories):
    raise RuntimeError("❌ 하나 이상의 카테고리가 올바르게 등록되지 않았습니다.")

# 🧠 분석 함수: 기존 + 사용자 정의 + 합산
def analyze_and_combine_relational_bonding(text):
    result = lexicon.analyze(
        str(text),
        categories=["leader", "family", "reddit_relational_bonding"],
        normalize=True
    )
    base_score = sum([
        result.get('leader', 0.0),
        result.get('family', 0.0)
    ])
    reddit_score = result.get('reddit_relational_bonding', 0.0)
    combined_score = base_score + reddit_score
    return pd.Series({
        "relational_bonding_base_score": round(base_score, 4),
        "reddit_relational_bonding_score": round(reddit_score, 4),
        "combined_relational_bonding_score": round(combined_score, 4)
    })

# 🏃 분석 실행
tqdm.pandas()
relational_df = df['comment'].progress_apply(analyze_and_combine_relational_bonding)

# 🔗 데이터 합치기
result_df = pd.concat([df, relational_df], axis=1)

# 💾 결과 저장
output_file = filename.replace(".xlsx", "_relational_bonding_combined_empath_final.xlsx")
result_df.to_excel(output_file, index=False, engine='openpyxl')
files.download(output_file)

print("✅ 완료: 최종 확장 사용자 정의 단어를 반영한 Relational Bonding 점수가 계산되어 저장되었습니다.")

affective connectedness

In [ ]:
# 📦 라이브러리 설치
!pip install empath pandas tqdm openpyxl --quiet

# 📚 라이브러리 불러오기
import pandas as pd
from empath import Empath
from tqdm import tqdm
from google.colab import files

# ⬆️ 데이터 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 필수 컬럼 체크
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 데이터에 없습니다!")
if df['comment'].isnull().all():
    raise ValueError("❌ 'comment' 열에 분석할 데이터가 없습니다!")

print(f"✅ 총 댓글 수: {len(df)}개")

# 🤖 Empath 로드
lexicon = Empath()

# 🔷 기존 Affective Connectedness 카테고리 단어 집합 가져오기
base_words = set(lexicon.cats['emotional']) \
           | set(lexicon.cats['affection']) \
           | set(lexicon.cats['joy']) \
           | set(lexicon.cats['sadness']) \
           | set(lexicon.cats['suffering'])

print(f"✅ 기존 Affective Connectedness 단어 수: {len(base_words)}")

# 🔷 Reddit 특화 Affective Connectedness 단어 리스트
reddit_affective_connectedness_words = [
    # … (너가 작성한 리스트 그대로)
]

# 🔷 중복 제거 & 정렬
reddit_affective_connectedness_words_unique = sorted({
    word for word in reddit_affective_connectedness_words if word not in base_words
})

print(f"✅ 중복 제거 후 사용자 정의 단어 수: {len(reddit_affective_connectedness_words_unique)}")
print(reddit_affective_connectedness_words_unique)

# 🔷 사용자 정의 카테고리 생성
lexicon.create_category("reddit_affective_connectedness", reddit_affective_connectedness_words_unique)
print("✅ 사용자 정의 카테고리 'reddit_affective_connectedness' 생성 완료!")

# 🔷 카테고리 정상 등록 테스트
test_result = lexicon.analyze("sending love and warm hugs", categories=[
    "emotional", "affection", "joy", "sadness", "suffering", "reddit_affective_connectedness"
])
expected_categories = ["emotional", "affection", "joy", "sadness", "suffering", "reddit_affective_connectedness"]
if not all(cat in test_result for cat in expected_categories):
    raise RuntimeError("❌ 하나 이상의 카테고리가 올바르게 등록되지 않았습니다.")

# 🧠 분석 함수: 기존 + 사용자 정의 + 합산
def analyze_and_combine_affective_connectedness(text):
    result = lexicon.analyze(
        str(text),
        categories=["emotional", "affection", "joy", "sadness", "suffering", "reddit_affective_connectedness"],
        normalize=True
    )
    base_score = sum([
        result.get('emotional', 0.0),
        result.get('affection', 0.0),
        result.get('joy', 0.0),
        result.get('sadness', 0.0),
        result.get('suffering', 0.0)
    ])
    reddit_score = result.get('reddit_affective_connectedness', 0.0)
    combined_score = base_score + reddit_score
    return pd.Series({
        "affective_connectedness_base_score": round(base_score, 4),
        "reddit_affective_connectedness_score": round(reddit_score, 4),
        "combined_affective_connectedness_score": round(combined_score, 4)
    })

# 🏃 분석 실행
tqdm.pandas()
affective_df = df['comment'].progress_apply(analyze_and_combine_affective_connectedness)

# 🔗 데이터 합치기
result_df = pd.concat([df, affective_df], axis=1)

# 💾 결과 저장
output_file = filename.replace(".xlsx", "_affective_connectedness_combined_empath.xlsx")
result_df.to_excel(output_file, index=False, engine='openpyxl')
files.download(output_file)

print("✅ 완료: 기존 + 사용자 정의 Affective Connectedness 점수가 계산되어 저장되었습니다.")

## 2. Mediator (self-disclosure)

In [ ]:
# 1. 환경 설정
!pip install pandas openpyxl nltk

import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # Colab 메모리 초과 방지

import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from google.colab import files

nltk.download('wordnet')
nltk.download('omw-1.4')

# 2. 파일 업로드
print("👉 댓글 데이터 (.xlsx) 업로드하세요")
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
df = pd.read_excel(file_path)
df['comment'] = df['comment'].astype(str)

# 3. 단어 수 필터링
df['word_count'] = df['comment'].apply(lambda x: len(re.findall(r'\b\w+\b', x.lower())))
df = df[df['word_count'] >= 10].copy()

# 4. LIWC-lite 사전 정의
liwc_dict = {
    "i": ["i"], "me": ["i"], "my": ["i"], "mine": ["i"], "myself": ["i"],
    "i'm": ["i"], "i’ll": ["i"], "i’ve": ["i"], "i’d": ["i"],
    "feel": ["affect"], "felt": ["affect"], "emotion": ["affect"], "emotions": ["affect"],
    "emotional": ["affect"], "feelings": ["affect"],
    "love": ["affect", "posemo"], "like": ["affect", "posemo"], "good": ["affect", "posemo"],
    "great": ["affect", "posemo"], "awesome": ["affect", "posemo"], "amazing": ["affect", "posemo"],
    "fun": ["affect", "posemo"], "better": ["affect", "posemo"], "enjoy": ["affect", "posemo"],
    "happy": ["affect", "posemo"],
    "sad": ["affect", "negemo"], "angry": ["affect", "negemo"], "lonely": ["affect", "negemo"],
    "anxious": ["affect", "negemo"], "scared": ["affect", "negemo"], "cry": ["affect", "negemo"],
    "abuse": ["affect", "negemo"], "paranoid": ["affect", "negemo"], "worry": ["affect", "negemo"],
    "depressed": ["affect", "negemo"],
    "think": ["cogproc"], "know": ["cogproc"], "understand": ["cogproc"],
    "guess": ["cogproc"], "realize": ["cogproc"], "believe": ["cogproc"],
    "consider": ["cogproc"], "aware": ["cogproc"], "want": ["cogproc"],
    "make": ["cogproc"], "need": ["cogproc"], "real": ["cogproc"],
    "talk": ["social"], "talking": ["social"], "friend": ["social"],
    "relationship": ["social"], "partner": ["social"], "chat": ["social"],
    "connect": ["social"], "people": ["social"], "app": ["social"],
    "replika": ["social"],
    "tired": ["bio"], "sick": ["bio"], "pain": ["bio"], "health": ["bio"],
    "sleep": ["bio"], "energy": ["bio"], "hungry": ["bio"], "mental": ["bio"]
}
liwc_categories = sorted(set(cat for cats in liwc_dict.values() for cat in cats))

# 5. 점수 계산
lemmatizer = WordNetLemmatizer()
def analyze_liwc_lemma(text):
    tokens = re.sub(r"[^\w\s]", "", text.lower()).split()
    lemmas = [lemmatizer.lemmatize(tok) for tok in tokens]
    counts = dict.fromkeys(liwc_categories, 0)
    total = len(lemmas)
    for tok in lemmas:
        if tok in liwc_dict:
            for cat in liwc_dict[tok]:
                counts[cat] += 1
    return [counts[cat] / total if total else 0.0 for cat in liwc_categories]

liwc_vectors = df['comment'].apply(analyze_liwc_lemma).tolist()
X = np.array(liwc_vectors)

target_cats = ['i', 'affect', 'negemo', 'cogproc', 'bio', 'social']
target_indices = [liwc_categories.index(cat) for cat in target_cats]
df['disclosure_score'] = X[:, target_indices].mean(axis=1)

# 6. 저장 및 다운로드
output_file = "replika_disclosure_score_filtered.xlsx"
df.to_excel(output_file, index=False)
files.download(output_file)

## 3. Moderator (subjectivity)

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files
import torch

# ✅ GPU 사용 여부 확인 및 디바이스 설정
device = 0 if torch.cuda.is_available() else -1
print("🚀 Using device:", "GPU" if device == 0 else "CPU")

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다. 올바른 열 이름인지 확인해주세요!")

# 🤖 Zero-shot 분류기 로드 (GPU 지정)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# 🎯 평가 항목 라벨 (Subjectivity만)
labels = ["Subjectivity"]

# 🧩 Subjectivity 점수 추출 함수
def classify_Subjectivity(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0
    try:
        result = classifier(comment, candidate_labels=labels)
        score = round(result["scores"][0], 4)
        return score
    except Exception as e:
        return 0.0

# 🏃 분석 실행 (진행률 표시)
tqdm.pandas()
df['Subjectivity_score'] = df['comment'].progress_apply(classify_Subjectivity)

# 💾 결과 저장 및 다운로드
output_filename = filename.replace(".xlsx", "_Subjectivity_score.xlsx")
df.to_excel(output_filename, index=False)
files.download(output_filename)

## 4. Dependent Variable (sentiment)

In [ ]:
# 📦 라이브러리 설치
!pip install scipy transformers pandas openpyxl tqdm matplotlib datasets scikit-learn --quiet

In [ ]:
# 📚 라이브러리
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm
from google.colab import files

# ✅ 디바이스 설정 (GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📂 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 데이터 로딩
df = pd.read_excel(filename)
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 없습니다.")
df['Review'] = df['comment'].fillna("")

# 🤖 모델 로딩 (사전 로딩 캐시 활용)
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
model.eval()

# ✅ 감성 분석 함수 (최적화: 고정 길이, GPU 직접 입력)
def analyze_sentiment_batch(texts):
    encoded = tokenizer(
        texts,
        return_tensors="pt",
        padding="max_length",     # 속도 증가
        truncation=True,
        max_length=128,
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        output = model(**encoded)
    probs = softmax(output.logits.cpu().numpy(), axis=1)
    return probs

# ✅ 배치 분석 (큰 배치 처리로 속도 향상)
batch_size = 64   # 더 크면 속도 ↑ (메모리 부족 시 32로)
scores = []

for i in tqdm(range(0, len(df), batch_size), desc="Analyzing"):
    batch = df['Review'].iloc[i:i+batch_size].tolist()
    try:
        probs = analyze_sentiment_batch(batch)
        for p in probs:
            pos, neu, neg = p[2], p[1], p[0]
            compound = round((pos - neg) * (1 - neu), 4)
            scores.append(compound)
    except Exception as e:
        print(f"❌ Error at batch {i}: {e}")
        scores.extend([0.0] * len(batch))

# ✅ 결과 저장 및 다운로드
df['sentiment_score'] = scores
output_file = filename.replace(".xlsx", "_roberta_sentiment_score_optimized.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 최적화된 RoBERTa 감성 분석 완료! 🚀")